In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

In [7]:
data_path = Path("C:/Users/hp/Downloads/marketing_campaign.xls")

In [9]:
df = pd.read_csv(data_path)

In [11]:
print("Shape:", df.shape)
print("\nDtypes:\n", df.dtypes)
print("\nMissing values (%):\n", (df.isnull().mean()*100).round(2))
print("\nDuplicate rows:", df.duplicated().sum())

Shape: (2240, 1)

Dtypes:
 ID\tYear_Birth\tEducation\tMarital_Status\tIncome\tKidhome\tTeenhome\tDt_Customer\tRecency\tMntWines\tMntFruits\tMntMeatProducts\tMntFishProducts\tMntSweetProducts\tMntGoldProds\tNumDealsPurchases\tNumWebPurchases\tNumCatalogPurchases\tNumStorePurchases\tNumWebVisitsMonth\tAcceptedCmp3\tAcceptedCmp4\tAcceptedCmp5\tAcceptedCmp1\tAcceptedCmp2\tComplain\tZ_CostContact\tZ_Revenue\tResponse    object
dtype: object

Missing values (%):
 ID\tYear_Birth\tEducation\tMarital_Status\tIncome\tKidhome\tTeenhome\tDt_Customer\tRecency\tMntWines\tMntFruits\tMntMeatProducts\tMntFishProducts\tMntSweetProducts\tMntGoldProds\tNumDealsPurchases\tNumWebPurchases\tNumCatalogPurchases\tNumStorePurchases\tNumWebVisitsMonth\tAcceptedCmp3\tAcceptedCmp4\tAcceptedCmp5\tAcceptedCmp1\tAcceptedCmp2\tComplain\tZ_CostContact\tZ_Revenue\tResponse    0.0
dtype: float64

Duplicate rows: 0


In [13]:
df.columns = (
    df.columns.astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r"[ \-]+", "_", regex=True)
              .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
)
df.head()

,idyear_birtheducationmarital_statusincomekidhometeenhomedt_customerrecencymntwinesmntfruitsmntmeatproductsmntfishproductsmntsweetproductsmntgoldprodsnumdealspurchasesnumwebpurchasesnumcatalogpurchasesnumstorepurchasesnumwebvisitsmonthacceptedcmp3acceptedcmp4acceptedcmp5acceptedcmp1acceptedcmp2complainz_costcontactz_revenueresponse
0,5524\t1957\tGraduation\tSingle\t58138\t0\t0\t0...
1,2174\t1954\tGraduation\tSingle\t46344\t1\t1\t0...
2,4141\t1965\tGraduation\tTogether\t71613\t0\t0\...
3,6182\t1984\tGraduation\tTogether\t26646\t1\t0\...
4,5324\t1981\tPhD\tMarried\t58293\t1\t0\t19-01-2...


In [17]:
# Drop duplicates
df = df.drop_duplicates()

# Numeric imputation
for c in df.select_dtypes(include=np.number).columns:
    df[c] = df[c].fillna(df[c].median())

# Categorical imputation
for c in df.select_dtypes(include=["object", "category"]).columns:
    mode = df[c].mode(dropna=True)
    if len(mode) > 0:
        df[c] = df[c].fillna(mode.iloc[0])


In [19]:
# Age
if "year_birth" in df.columns:
    df["age"] = datetime.now().year - df["year_birth"]
    df.loc[(df["age"] < 0) | (df["age"] > 120), "age"] = np.nan

# Total spent
mnt_cols = [c for c in df.columns if c.startswith("mnt")]
if mnt_cols:
    df["total_spent"] = df[mnt_cols].sum(axis=1)

# Total purchases
purchase_cols = [c for c in df.columns if "purchases" in c or "purchase" in c or "visits" in c]
if purchase_cols:
    df["total_purchases"] = df[purchase_cols].sum(axis=1)

# Family size
if "kidhome" in df.columns and "teenhome" in df.columns:
    df["family_size"] = df["kidhome"].astype(int) + df["teenhome"].astype(int) + 1

In [21]:
for c in df.select_dtypes(include=np.number).columns:
    q1 = df[c].quantile(0.25)
    q3 = df[c].quantile(0.75)
    iqr = q3 - q1
    if iqr > 0:
        lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
        df[c] = df[c].clip(lower, upper)

In [23]:
cat_cols = [c for c in df.select_dtypes(include=["object", "category"]).columns if df[c].nunique() <= 20]
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [25]:
df.to_csv("marketing_campaign_cleaned.csv", index=False)
print("✅ Cleaning done! Saved as marketing_campaign_cleaned.csv")
df.head()

✅ Cleaning done! Saved as marketing_campaign_cleaned.csv


,idyear_birtheducationmarital_statusincomekidhometeenhomedt_customerrecencymntwinesmntfruitsmntmeatproductsmntfishproductsmntsweetproductsmntgoldprodsnumdealspurchasesnumwebpurchasesnumcatalogpurchasesnumstorepurchasesnumwebvisitsmonthacceptedcmp3acceptedcmp4acceptedcmp5acceptedcmp1acceptedcmp2complainz_costcontactz_revenueresponse,total_purchases
0,5524\t1957\tGraduation\tSingle\t58138\t0\t0\t0...,5524\t1957\tGraduation\tSingle\t58138\t0\t0\t0...
1,2174\t1954\tGraduation\tSingle\t46344\t1\t1\t0...,2174\t1954\tGraduation\tSingle\t46344\t1\t1\t0...
2,4141\t1965\tGraduation\tTogether\t71613\t0\t0\...,4141\t1965\tGraduation\tTogether\t71613\t0\t0\...
3,6182\t1984\tGraduation\tTogether\t26646\t1\t0\...,6182\t1984\tGraduation\tTogether\t26646\t1\t0\...
4,5324\t1981\tPhD\tMarried\t58293\t1\t0\t19-01-2...,5324\t1981\tPhD\tMarried\t58293\t1\t0\t19-01-2...


In [27]:
df.isnull().sum()

idyear_birtheducationmarital_statusincomekidhometeenhomedt_customerrecencymntwinesmntfruitsmntmeatproductsmntfishproductsmntsweetproductsmntgoldprodsnumdealspurchasesnumwebpurchasesnumcatalogpurchasesnumstorepurchasesnumwebvisitsmonthacceptedcmp3acceptedcmp4acceptedcmp5acceptedcmp1acceptedcmp2complainz_costcontactz_revenueresponse    0
total_purchases                                                                                                                                                                                                                                                                                                                                 0
dtype: int64